In [1]:
# ============================================================
# FINAL PLAYER + BALL TRACKING (STABLE, TEAM-PERSISTENT)
# ============================================================

import cv2
import numpy as np
import torch
from ultralytics import YOLO
from collections import deque

# ------------------------------
# DEVICE & MODEL
# ------------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

yolo_model = YOLO("yolov8n.pt")
yolo_model.to(DEVICE)

PERSON_ID = 0
BALL_ID = 32

# ------------------------------
# MANUAL GEOMETRY (EDIT)
# ------------------------------
COURT_POLY = np.array([
     [313, 292],
     [996, 289],
     [1198, 708],
     [125, 707],
], dtype=np.int32)

SPLIT_Y = 443   # horizontal net line

# ------------------------------
# VIDEO I/O
# ------------------------------
VIDEO_PATH = "/home/naman/Task-0/volleyball_match.mp4"
OUTPUT_PATH = "/home/naman/Task-0/volleyball_output.mp4"

cap = cv2.VideoCapture(VIDEO_PATH)
FPS = int(cap.get(cv2.CAP_PROP_FPS))
W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
FRAME_AREA = W * H

out = cv2.VideoWriter(
    OUTPUT_PATH,
    cv2.VideoWriter_fourcc(*"mp4v"),
    FPS,
    (W, H)
)

# ------------------------------
# COURT MASK
# ------------------------------
def apply_court_mask(frame):
    mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [COURT_POLY], 255)
    return cv2.bitwise_and(frame, frame, mask=mask)

# ------------------------------
# SIMPLE IOU TRACKER (PLAYERS)
# ------------------------------
def iou(a, b):
    xA = max(a[0], b[0])
    yA = max(a[1], b[1])
    xB = min(a[2], b[2])
    yB = min(a[3], b[3])
    inter = max(0, xB - xA) * max(0, yB - yA)
    areaA = (a[2] - a[0]) * (a[3] - a[1])
    areaB = (b[2] - b[0]) * (b[3] - b[1])
    return inter / (areaA + areaB - inter + 1e-6)

players = {}        # id → dict(bbox, team, miss)
NEXT_ID = 0
MAX_MISS = 15

# ------------------------------
# BALL KALMAN FILTER
# ------------------------------
kalman = cv2.KalmanFilter(4, 2)
kalman.measurementMatrix = np.array([[1,0,0,0],[0,1,0,0]], np.float32)
kalman.transitionMatrix = np.array([[1,0,1,0],[0,1,0,1],[0,0,1,0],[0,0,0,1]], np.float32)
kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03

ball_trail = deque(maxlen=50)

# ------------------------------
# SAFETY
# ------------------------------
MAX_FRAMES = 2000
frame_count = 0

# ------------------------------
# MAIN LOOP
# ------------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count > MAX_FRAMES:
        break

    court_frame = apply_court_mask(frame)

    results = yolo_model(
        court_frame,
        conf=0.15,
        device=DEVICE,
        verbose=False
    )

    detections = []
    ball_measurement = None

    # ---- PARSE YOLO OUTPUT ----
    for r in results:
        if r.boxes is None:
            continue
        for box in r.boxes:
            cls = int(box.cls[0])
            x1,y1,x2,y2 = box.xyxy[0].cpu().numpy().astype(int)
            cx = (x1 + x2)//2
            cy = (y1 + y2)//2

            if cls == PERSON_ID:
                detections.append((x1,y1,x2,y2,cx,cy))

            elif cls == BALL_ID:
                area = (x2-x1)*(y2-y1)
                if 50 < area < 0.01*FRAME_AREA:
                    ball_measurement = np.array([[np.float32(cx)], [np.float32(cy)]])
                    cv2.circle(frame, (cx,cy), 6, (0,255,255), -1)

    # --------------------------
    # PLAYER TRACKING
    # --------------------------
    updated = {}

    for det in detections:
        x1,y1,x2,y2,cx,cy = det
        best_iou, best_id = 0, None

        for pid, pdata in players.items():
            i = iou((x1,y1,x2,y2), pdata["bbox"])
            if i > best_iou:
                best_iou, best_id = i, pid

        if best_iou > 0.3:
            players[best_id]["bbox"] = (x1,y1,x2,y2)
            players[best_id]["miss"] = 0
            updated[best_id] = True
        else:
            team = "A" if cy > SPLIT_Y else "B"
            players[NEXT_ID] = {
                "bbox": (x1,y1,x2,y2),
                "team": team,
                "miss": 0
            }
            updated[NEXT_ID] = True
            NEXT_ID += 1

    for pid in list(players.keys()):
        if pid not in updated:
            players[pid]["miss"] += 1
            if players[pid]["miss"] > MAX_MISS:
                del players[pid]

    # --------------------------
    # BALL TRACKING
    # --------------------------
    if ball_measurement is not None:
        kalman.correct(ball_measurement)

    pred = kalman.predict()
    px, py = int(pred[0]), int(pred[1])
    ball_trail.append((px, py))

    for i in range(1, len(ball_trail)):
        cv2.line(frame, ball_trail[i-1], ball_trail[i], (0,255,255), 2)

    # --------------------------
    # DRAW PLAYERS
    # --------------------------
    for pid, pdata in players.items():
        x1,y1,x2,y2 = pdata["bbox"]
        team = pdata["team"]
        color = (0,255,0) if team=="A" else (255,0,0)
        cv2.rectangle(frame, (x1,y1), (x2,y2), color, 2)
        cv2.putText(frame, f"ID {pid} | Team {team}",
                    (x1, y1-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    cv2.polylines(frame, [COURT_POLY], True, (255,255,255), 2)
    cv2.line(frame, (0,SPLIT_Y), (W,SPLIT_Y), (0,0,255), 2)

    out.write(frame)

cap.release()
out.release()
print("DONE. Output saved:", OUTPUT_PATH)


Using device: cuda


/tmp/ipykernel_5746/3224615012.py:179: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  px, py = int(pred[0]), int(pred[1])


DONE. Output saved: /home/naman/Task-0/volleyball_output.mp4
